In [1]:
import requests
from bs4 import BeautifulSoup
import csv

print("HI")

HI


In [2]:
 url = 'https://bestpitchdeck.com/tomorrow-health'
 response = requests.get(url)
 page = response.content
 soup = BeautifulSoup(page,'lxml')

In [3]:
pitchs = soup.find("div", {'class': 'col-11 col-lg-5'})

def get_card_info(pitchs):
    discription = pitchs.contents[0]

 
    print('1------- ',pitchs.contents[1])
    print("2-------- ",pitchs.contents[2])
get_card_info(pitchs)    

1-------  <div class="mt-3">
<h1 class="text-primary">Tomorrow Health Pitch Deck</h1>
<p class="text-muted mb-2">Tomorrow Health is a technology-driven home healthcare company that focuses on the way home-based care is ordered, delivered, and paid for. The company was founded by Vijay Kedar in 2020.  Tomorrow Health is working with more than 125 health plans and hospital systems in 29 states; a recent contract will see the company deliver home medical equipment and supplies to Geisinger Health Plan’s more than 500,000 customers.<br/>Tomorrow Health announced it had raised a $60 million Series B funding round led by Bond, with participation from Andreessen Horowitz, Obvious Ventures, BoxGroup, and Sound Ventures. Andreessen Horowitz also led the company's $25 million Series A round. The startup has raised $92.5 million since its founding in 2017.</p>
<p class="lead text-muted mb-2">2022 • Series B  <i class="fa fa-hand-holding-usd"></i></p>
</div>
2--------  



In [4]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_data():
    base_url = 'https://bestpitchdeck.com'
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    decks = []
    pitchs = soup.find_all("div", {'class': 'card'})
    
    for pitch in pitchs:
        card_body = pitch.find('div', class_='card-body')
        card_footer = pitch.find('div', class_='card-footer')
        
        if not card_body or not card_footer:
            continue
        
        title_tag = card_body.find('h5')
        company_name = title_tag.text.strip() if title_tag else ''
        
        description_tag = card_body.find('p', class_='card-text')
        description = description_tag.text.strip() if description_tag else ''
        
        details_tag = card_body.find('small', class_='text-semi-op')
        details = details_tag.text.strip().split('•') if details_tag else []
        
        amount_raised = details[0].strip() if len(details) > 0 else ''
        year = details[1].strip() if len(details) > 1 else ''
        stage = details[2].strip() if len(details) > 2 else ''
        
        business_type = ''
        footer_tag = card_footer.find('strong', class_='text-semi-op')
        if footer_tag and 'Business:' in footer_tag.text:
            business_type = footer_tag.next_sibling.strip() if footer_tag.next_sibling else ''
        
        img_tag = title_tag.find('img') if title_tag else None
        img_url = img_tag['src'] if img_tag else ''
        
        view_deck_tag = card_body.find('a', href=True)
        detail_url = base_url + view_deck_tag['href'] if view_deck_tag else ''
        
        # Scrape additional information from the detail page
        additional_info = scrape_detail_page(detail_url)
        
        decks.append([company_name, description, amount_raised, year, stage, business_type, img_url, detail_url] + additional_info)
    
    return decks

def scrape_detail_page(url):
    if not url:
        return [''] * 13  # Adjust the number of empty strings based on the number of additional fields you want to scrape
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Scrape additional details
    full_description = get_text(soup.select_one('.col-11.col-lg-5 .text-muted.mb-2'))
    pdf_url = soup.select_one('iframe')['src'] if soup.select_one('iframe') else ''
    
    about_section = soup.find('div', class_='row py-5')
    raised_funds = get_text_from_li(about_section, 'li:nth-child(1)')
    year = get_text_from_li(about_section, 'li:nth-child(2)')
    stage = get_text_from_li(about_section, 'li:nth-child(3)')
    investors = get_text_from_li(about_section, 'li:nth-child(4)')
    
    about_company_section = about_section.find_all('div', class_='col-12 col-md-6') if about_section else []
    about_company_info = about_company_section[1] if len(about_company_section) > 1 else None
    about = get_text(about_company_info.find('p', class_='text-muted mb-2') if about_company_info else None)
    industry = get_text_from_li(about_company_info, 'li:nth-child(1)')
    tags = get_text_from_li(about_company_info, 'li:nth-child(2)')
    business_model = get_text_from_li(about_company_info, 'li:nth-child(3)')
    customer_model = get_text_from_li(about_company_info, 'li:nth-child(4)')
    website = get_text_from_li(about_company_info, 'li:nth-child(5)')
    legal_name = get_text_from_li(about_company_info, 'li:nth-child(6)')
    
    return [
        full_description, year, stage, pdf_url, investors, raised_funds,
        about, industry, tags, business_model, customer_model, website, legal_name
    ]

def get_text(element):
    return element.text.strip() if element else ''

def get_text_from_li(section, selector):
    element = section.select_one(selector) if section else None
    return element.text.split(':')[1].strip() if element and ':' in element.text else ''

def save_to_csv(decks, filename='pitch_decks.csv'):
    header = [
        'Company Name', 'Description', 'Amount Raised (Main)', 'Year (Main)', 'Stage (Main)', 'Business Type',
        'Image URL', 'Detail URL', 'Full Description', 'Year', 'Stage', 'PDF URL', 'Investors', 'Raised Funds',
        'About', 'Industry', 'Tags', 'Business Model', 'Customer Model', 'Website', 'Legal Name'
    ]
    
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(decks)

decks = scrape_data()
save_to_csv(decks)
print("Data has been saved to pitch_decks.csv")


Data has been saved to pitch_decks.csv


In [5]:
import pandas
df = pandas.read_csv('/kaggle/working/pitch_decks.csv')
print(df.columns)


Index(['Company Name', 'Description', 'Amount Raised (Main)', 'Year (Main)',
       'Stage (Main)', 'Business Type', 'Image URL', 'Detail URL',
       'Full Description', 'Year', 'Stage', 'PDF URL', 'Investors',
       'Raised Funds', 'About', 'Industry', 'Tags', 'Business Model',
       'Customer Model', 'Website', 'Legal Name'],
      dtype='object')


In [6]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1485 entries, 0 to 1484
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Company Name          1485 non-null   object
 1   Description           1475 non-null   object
 2   Amount Raised (Main)  1443 non-null   object
 3   Year (Main)           1472 non-null   object
 4   Stage (Main)          1451 non-null   object
 5   Business Type         1447 non-null   object
 6   Image URL             1307 non-null   object
 7   Detail URL            1485 non-null   object
 8   Full Description      1361 non-null   object
 9   Year                  1468 non-null   object
 10  Stage                 1452 non-null   object
 11  PDF URL               1475 non-null   object
 12  Investors             550 non-null    object
 13  Raised Funds          1441 non-null   object
 14  About                 1464 non-null   object
 15  Industry              1478 non-null   

In [7]:
print(df.head())
print(df.tail())


  Company Name Description Amount Raised (Main) Year (Main) Stage (Main)  \
0      Mistral         NaN                €105M        2023         Seed   
1      Shopify         NaN                 $66M        2016   Late Stage   
2        Tesla         NaN                $192M        2017   Late Stage   
3       WeWork         NaN                $335M        2014   Late Stage   
4       Airbnb         NaN                $600K        2008         Seed   

  Business Type                              Image URL  \
0           B2C                 /img/logos/mistral.png   
1           B2B  https://logo.clearbit.com/shopify.com   
2           B2C                   /img/logos/tesla.png   
3           B2B                  /img/logos/wework.jpg   
4      B2C, B2B                  /img/logos/airbnb.png   

                          Detail URL  \
0  https://bestpitchdeck.com/mistral   
1  https://bestpitchdeck.com/shopify   
2    https://bestpitchdeck.com/tesla   
3   https://bestpitchdeck.com/wewo

In [8]:
print(df.isnull().sum())


Company Name              0
Description              10
Amount Raised (Main)     42
Year (Main)              13
Stage (Main)             34
Business Type            38
Image URL               178
Detail URL                0
Full Description        124
Year                     17
Stage                    33
PDF URL                  10
Investors               935
Raised Funds             44
About                    21
Industry                  7
Tags                      7
Business Model          604
Customer Model          617
Website                  48
Legal Name               19
dtype: int64


In [9]:

columns_to_drop = ['Detail URL']
df = df.drop(columns=columns_to_drop)

# Rename specified columns
df = df.rename(columns={
    'Amount Raised (Main)': 'Amount Raised',
    'Year (Main)': 'Year',
    'Stage (Main)': 'Stage',
    'Business Type': 'Business Model'
})

# Verify the changes
print(df.columns)

Index(['Company Name', 'Description', 'Amount Raised', 'Year', 'Stage',
       'Business Model', 'Image URL', 'Full Description', 'Year', 'Stage',
       'PDF URL', 'Investors', 'Raised Funds', 'About', 'Industry', 'Tags',
       'Business Model', 'Customer Model', 'Website', 'Legal Name'],
      dtype='object')


In [10]:
df['Type'] = 'ST'

# Save DataFrame to CSV
filename = 'pitch_decks.csv'
df.to_csv(filename, index=False)
print(f"DataFrame successfully converted and saved")

DataFrame successfully converted and saved
